# Automated ML

Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.model import Model
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep
from azureml.automl.core.shared import constants
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice, AciWebservice
from azureml.core.environment import CondaDependencies


import joblib

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Dataset - Heart Failure Clinical Records Dataset

### Overview
Cardiovascular diseases (CVDs) are the number 1 cause of death globally, taking an estimated 17.9 million lives each year, which accounts for 31% of all deaths worlwide.
Heart failure is a common event caused by CVDs and this dataset contains 12 features that can be used to predict mortality by heart failure.

Most cardiovascular diseases can be prevented by addressing behavioural risk factors such as tobacco use, unhealthy diet and obesity, physical inactivity and harmful use of alcohol using population-wide strategies.

People with cardiovascular disease or who are at high cardiovascular risk (due to the presence of one or more risk factors such as hypertension, diabetes, hyperlipidaemia or already established disease) need early detection and management wherein a machine learning model can be of great help.

### Objective
Train a model to predict whether the person with Cardiovascular disease will survive or not

In [2]:
# Initialize Workspace
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-138775
aml-quickstarts-138775
southcentralus
9a7511b8-150f-4a58-8528-3e7d50216c31


In [3]:
# choose a name for experiment
experiment_name = 'automl_heart_failure_prediction'

if "automl" not in os.listdir():
    os.mkdir("./automl")

project_folder = './automl'

experiment=Experiment(ws, experiment_name)

In [4]:
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: quick-starts-ws-138775
Azure region: southcentralus
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-138775


## Create or Attach an AmlCompute cluster

In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

compute_cluster_name = "capstone-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace = ws, name = compute_cluster_name)
    print('Found existing cluster, use it.')
    
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)
    
    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.


## Data

In [6]:
dataset = Dataset.get_by_name(ws, name='Heart_Failure_Clinical_Records_Dataset')

df = dataset.to_pandas_dataframe()

df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [7]:
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## AutoML Configuration

Explain why you chose the automl settings and cofiguration you used below.

In [8]:
# Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4,
    "n_cross_validations": 5,
    "primary_metric" : 'accuracy'}

# Put your automl config here
automl_config = AutoMLConfig(compute_target = compute_target,
                             task = "classification",
                             training_data = dataset,
                             label_column_name = "DEATH_EVENT",
                             path = project_folder,
                             enable_early_stopping = True,
                             enable_onnx_compatible_models = True,
                             featurization = 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [9]:
# Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

The cell below uses `RunDetails` widget to show the different experiments.

In [10]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS

{'runId': 'AutoML_7f0d0b56-e736-418e-811b-5b2c59d6c18b',
 'target': 'capstone-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-02-13T10:30:06.269658Z',
 'endTimeUtc': '2021-02-13T10:53:43.411727Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'capstone-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"automl_heart_failure_prediction","subscription_id":"9a7511b8-150f-4a58-8528-3e7d50216c31","resource_group":"aml-quickstarts-138775","workspace_name":"quick-starts-ws-138775","region":"southcentralus","compute_target":"capstone-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"

## Best Model

Get the best model from the automl experiments and display all the properties of the model.

In [11]:
automl_best_run, automl_fitted_model = remote_run.get_output()

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


In [12]:
automl_best_run.get_metrics()

{'precision_score_macro': 0.8732408890805944,
 'f1_score_macro': 0.8420130918883105,
 'recall_score_weighted': 0.8696045197740112,
 'AUC_micro': 0.916415525551406,
 'average_precision_score_weighted': 0.9207461462475868,
 'AUC_weighted': 0.9130143964562569,
 'recall_score_macro': 0.8376446566998894,
 'average_precision_score_micro': 0.9181731752467938,
 'balanced_accuracy': 0.8376446566998894,
 'f1_score_micro': 0.8696045197740112,
 'average_precision_score_macro': 0.8946453712173341,
 'precision_score_weighted': 0.8823379253880681,
 'accuracy': 0.8696045197740112,
 'weighted_accuracy': 0.8909894276000188,
 'norm_macro_recall': 0.6752893133997786,
 'recall_score_micro': 0.8696045197740112,
 'matthews_correlation': 0.7082603007554998,
 'f1_score_weighted': 0.8639828127526371,
 'AUC_macro': 0.913014396456257,
 'precision_score_micro': 0.8696045197740112,
 'log_loss': 0.3824770658891571,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_7f0d0b56-e736-418e-811b-5b2c59d6c18b_

In [13]:
print(automl_best_run)

Run(Experiment: automl_heart_failure_prediction,
Id: AutoML_7f0d0b56-e736-418e-811b-5b2c59d6c18b_36,
Type: azureml.scriptrun,
Status: Completed)


In [14]:
print(automl_fitted_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                min_child_weight=2,
                                                                                                min_split_gain=0,
                                                                                                n_estimators=25,
                                                 

In [15]:
# Retrieve and save your best automl model.

os.makedirs('output', exist_ok=True)

joblib.dump(value = automl_fitted_model, filename = 'output/best-automl.pkl')

['output/best-automl.pkl']

In [16]:
model = Model.register(workspace = ws,
                        model_path = 'output/best-automl.pkl',
                        model_name = 'automl-best-model')

automl_best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'my-conda-env.yml')

Registering model automl-best-model


## Model Deployment

Register the model, create an inference config and deploy the model as a web service.

In [17]:
env = Environment.from_conda_specification('my-conda-env', 'my-conda-env.yml')

inference_config = InferenceConfig(entry_script='score.py', environment=env)

aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                memory_gb = 1,
                                                auth_enabled = True, 
                                                enable_app_insights = True,
                                                tags = {'name':'Heart failure prediction'},
                                                description='Heart failure prediction model')

service = model.deploy(workspace = ws, 
                       name = 'predict-heart-failure', 
                       models = [model], 
                       inference_config = inference_config,
                       deployment_config = aci_config)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...............................................................................................................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [18]:
print(service.state)

Healthy


In [ ]:
# print("scoring URI: " + service.scoring_uri)

# print("Swagger URI: " + service.swagger_uri)

# print("Authetication Key: " + service.get_keys()[0])

## Send a request to the web service you deployed to test it.

In [21]:
import requests
import json

scoringuri = service.scoring_uri
key = service.get_keys()[0]

data= { "data":
       [
           {
               'age': 74,
               'anaemia': 1,
               'creatinine_phosphokinase': 1618,
               'diabetes': 1,
               'ejection_fraction': 27,
               'high_blood_pressure': 1,
               'platelets': 275095,
               'serum_creatinine': 2.3,
               'serum_sodium': 133,
               'sex': 0,
               'smoking': 0,
               'time': 9
               
           },
           {
               'age': 46,
               'anaemia': 0,
               'creatinine_phosphokinase': 800,
               'diabetes': 0,
               'ejection_fraction': 48,
               'high_blood_pressure': 1,
               'platelets': 259000,
               'serum_creatinine': 1.79,
               'serum_sodium': 135,
               'sex': 1,
               'smoking': 0,
               'time': 107
           }
       ]
    }
input_data = json.dumps(data)

headers = {'Content-Type': 'application/json'}
headers['Authorization'] = f'Bearer {key}'

response = requests.post(scoringuri, input_data, headers = headers)
print(response.text)

[1, 0]


## Convert model to ONNX format

In [20]:
from azureml.automl.runtime.onnx_convert import OnnxConverter

automl_best_run_onnx, automl_fitted_model_onnx = remote_run.get_output(return_onnx_model=True)

OnnxConverter.save_onnx_model(automl_fitted_model_onnx, './output/best_automl_model.onnx')

## Print the logs of the web service and delete the service

In [22]:
print(service.get_logs())

/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-02-13T11:17:48,254299238+00:00 - iot-server/run 
2021-02-13T11:17:48,254782057+00:00 - rsyslog/run 
2021-02-13T11:17:48,256454021+00:00 - nginx/run 
2021-02-13T11:17:48,257948179+00:00 - gunicorn/run 
rsyslogd

In [23]:
service.delete()